In [ ]:
#This is required to import the APMAE model from the other directory
import sys, os
path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.dirname('./dimension_selection.ipynb'), os.path.pardir, 'Model')))
if (not (path2add in sys.path)) :
    sys.path.append(path2add)

In [ ]:
#Our code
from DataUtil.DataLoader import IterableAttentionLoader
from DataUtil.AttentionData import AttentionData
from ap_mae import APMAE

#Imported packages
from transformers import AutoModelForCausalLM

import numpy as np
from tqdm import tqdm
from collections import Counter

from cuml.metrics import trustworthiness

# Setup

In [ ]:
size = '3B' #Set the size of the target model here. 3B, 7B or 15B

db_name = "reproduction_{}".format(size)
target_model_name = 'bigcode/starcoder2-{}'.format(size.lower())
encoding_model_name = 'LaughingLogits/AP-MAE-SC2-{}'.format(size)
dataset_name = 'LaughingLogits/Stackless_Java_V2'
split = 'test'

device = 'cpu'
languages = ['java']

understanding = ["identifiers"]
literals = ["boolean_literals", "string_literals", "numeric_literals"]
operators = ['boolean_operators', 'mathematical_operators', 'assignment_operators']
syntax = ['eol', 'closing_bracket']
tasks = ["random"] + understanding + literals + operators + syntax

samples_per_task = 1000
context_length = 256

In [ ]:
# These can be replaced with a list of keys, but we used all values in our investigation
# e.g. layers = ['0', 1'] for all values in layer 0 and 1
langs = "*"
corrects = "*"
querys = "*"
layers = "*"
heads = "*"

In [ ]:
target_model = AutoModelForCausalLM.from_pretrained(target_model_name, device_map="auto")
attention_data = AttentionData(target_model.config, tasks, languages, db_name)

# Target Dimension Selection

In [ ]:
X = attention_data.data.get_grouped_samples(langs, corrects, querys, layers, heads, 'enc_cls')
for dims in [2,4,6,8]:
    for trial in range(5):
        X_reduced = UMAP(n_components=dims, min_dist=0, metric = 'manhattan').fit_transform(X)
        for n in range(10):
            num_sample = 1000000
            select = [True] * num_sample
            ignore = [False] * (len(X)-num_sample)
            select = select+ignore
            shuffle(select)


            trust = trustworthiness(X[select], X_reduced[select], batch_size=256, n_neighbors=1000)
            with open("./trust_d" + str(dims) + "_n" + str(trial) + ".out", "a") as outfile:
                outfile.write(str(trust) + "\n")